In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from matplotlib import pyplot as plt

from PIL import Image 
import cv2
import numpy as np
import time

time_start=time.time()
print("time_start=", time_start)

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pymysql
db = pymysql.connect(host = 'mydbinstance.ctt6qfpp9mdh.eu-west-3.rds.amazonaws.com',
                     port = 3306,
                     user = 'husiyu',
                     passwd = 'husiyu123',
                     database = 'dbname',
                     charset = 'utf8')

cur = db.cursor()
sql = "SELECT key_file FROM dbname.keyf where nur=1;"
cur.execute(sql)
id_trainer = cur.fetchone()[0]
cur.execute(sql)
db.commit()
cur.close()
db.close()

gauth = GoogleAuth()
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file2 = drive.CreateFile({'id': id_trainer})
file2.GetContentFile('trainer.yml')
base = BaseOverlay("base.bit")

# initilization: monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

# monitor (output) frame buffer size
frame_out_w = 640
frame_out_h = 480
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480
# create Fisherfaces recognizer
recognizer = cv2.face.FisherFaceRecognizer_create()
# load trainer file
recognizer.read('trainer.yml')
#import face cascade classifier
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)
font = cv2.FONT_HERSHEY_SIMPLEX

idnum = 0

names = ['HY', 'HSY']
# open camera
cam = cv2.VideoCapture(0)
#Frame rate
fps = cam.get(5)
#Number of frames
f = cam.get(7)
print("fps=",fps)
print("f=",f)
num = 0
#print(cam.read())
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
print("minW=",minW)
print("minH=",minH)

print("Capture device is open: " + str(cam.isOpened()))

while True:
    # read the image from the camera
    ret, img = cam.read()
    # convert to grayscale image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # face detection
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(int(minW), int(minH))
    )
    
    #draw the rectangular (bounding box)
    for (x, y, w, h) in faces:
        cv2.imwrite("demo.jpg", gray[y:y+h, x:x+w])
        im = Image.open(r"demo.jpg")  
        newsize = (200, 200) 
        im1 = im.resize(newsize) 
        im1.save("im1.jpg", 'JPEG', quality = 100)
        predict_image=cv2.imread("im1.jpg",cv2.IMREAD_GRAYSCALE)
        idnum, confidence = recognizer.predict(predict_image)
        num = num + 1
        
        # calculate the recognition time per image
        if num > 1:
            time_cost = (time.time() - time_s)/(num-1)
            print("time_cost=", time_cost)
        else:
            time_s=time.time()
            print("time_s=", time_s)
        
        if confidence < 3000:
            idnum = names[idnum]
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)     #color:BRG
        else:
            idnum = "unknown"
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
        # show id and confidence
        cv2.putText(img, str(idnum), (x+5, y-5), font, 1, (255, 0, 0), 1)
        cv2.putText(img, str(confidence), (x+5, y+h-5), font, 1, (0, 0, 0), 1)
       
    # Display webcam image via HDMI Out
    if (ret):      
        outframe = hdmi_out.newframe()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        outframe[0:480,0:640,:] = img[0:480,0:640,:]
        hdmi_out.writeframe(outframe)
    else:
        raise RuntimeError("Failed to read from camera.")
        
    cv2.imshow('video', img)
    
    k = cv2.waitKey(1) 
    if k == 27:
        break
        
# turn off the camera
cam.release()
cv2.destroyAllWindows()